-----
Build a Large Language Model
Sebastian Raschka
-----

# Working with text data

  - During the pretraining stage, LLMs process text, one word at a time.

## 2.1 - Understanding word embeddings

  - Converting data into a vector format is referred to as embedding.  An embedding is a mapping from discrete objects, such as words, images, or even entire documents, to point in a continuous vector space - the primary purpose of embeddings is to convert nonnumeric data into a format that neural networks can process.
  - In addition to word embeddings, there are also embedding for sentences, paragraphs, or whole documents.
  - Retrieval-augmented generation combines generation (like producting text) with retrieval (like searching an external knowledge base) to pull relevant information when generating text.

## 2.2 - Tokenizing text

  - We will tokenize "The Verdict," from https://en.wikisource.org/wiki/The_Verdict


In [2]:
# Download our text
import urllib.request
url = ("https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/refs/heads/main/ch02/01_main-chapter-code/the-verdict.txt")
file_path = "the-verdict.txt"
urllib.request.urlretrieve(url, file_path)

# Now load our text file
with open(file_path, "r", encoding="utf-8") as file:
    raw_text = file.read()
print("Total number of characters:", len(raw_text))
print(raw_text[:99])

Total number of characters: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 
